<a href="https://colab.research.google.com/github/kbansal17/flood_mapping_in_assam/blob/main/cloudmainproject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install earthengine-api


In [ ]:
!pip install geemap  # Optional: For visualizing maps


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 11.1 MB/s eta 0:00:00


In [ ]:
import ee

# Authenticate with your Google account
ee.Authenticate()

# Initialize the Earth Engine library with a specified project ID
ee.Initialize(project='ee-khushibansal17june2003')  # Replace with your project ID


In [ ]:
# Define Goalpara district using a FeatureCollection from India boundaries.
goalpara = ee.FeatureCollection("FAO/GAUL/2015/level2") \
            .filter(ee.Filter.eq('ADM2_NAME', 'Goalpara')) \
            .filter(ee.Filter.eq('ADM1_NAME', 'Assam'))

# Visualize Goalpara region
import geemap
Map = geemap.Map()
Map.centerObject(goalpara, 9)
Map.addLayer(goalpara, {}, "Goalpara District")
Map


Map(center=[26.048206464191377, 90.60879118863485], controls=(WidgetControl(options=['position', 'transparent_…

In [ ]:
# Load Sentinel-1 SAR data (2015-2024) for Goalpara district.
sentinel1 = ee.ImageCollection('COPERNICUS/S1_GRD') \
             .filterBounds(goalpara) \
             .filterDate('2015-01-01', '2024-12-31') \
             .filter(ee.Filter.eq('instrumentMode', 'IW')) \
             .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \
             .filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))  # Use descending pass for better results.

print(f'Total Images Loaded: {sentinel1.size().getInfo()}')


Total Images Loaded: 257


In [ ]:
# Define dry season and flood season periods
pre_flood = sentinel1.filterDate('2023-01-01', '2023-03-31').median()
post_flood = sentinel1.filterDate('2023-07-01', '2023-09-30').median()

# Clip images to Goalpara region
pre_flood = pre_flood.clip(goalpara)
post_flood = post_flood.clip(goalpara)

# Display the images
Map.addLayer(pre_flood, {'min': -20, 'max': 0}, 'Pre-Flood VV')
Map.addLayer(post_flood, {'min': -20, 'max': 0}, 'Post-Flood VV')
Map


Map(bottom=56007.0, center=[26.048206464191377, 90.60879118863485], controls=(WidgetControl(options=['position…

In [ ]:
# Print band names to see what bands the images contain
print(pre_flood.bandNames().getInfo())
print(post_flood.bandNames().getInfo())


['VV', 'VH', 'angle']
['VV', 'VH', 'angle']


In [ ]:
# Select only the VV band for pre-flood and post-flood images
pre_flood = pre_flood.select('VV')
post_flood = post_flood.select('VV')

# Calculate the difference (delta) between pre-flood and post-flood VV backscatter
flood_map = pre_flood.subtract(post_flood).rename('flood_extent')


In [ ]:
# Load Sentinel-2 data (optional, for cross-validation)
sentinel2 = ee.ImageCollection('COPERNICUS/S2') \
    .filterBounds(goalpara) \
    .filterDate('2023-07-01', '2023-09-30') \
    .median() \
    .clip(goalpara)

# Visualize Sentinel-2 imagery for comparison
Map.addLayer(sentinel2, {'bands': ['B8', 'B4', 'B3'], 'min': 0, 'max': 3000},
             'Sentinel-2 (Flood Season)')
Map


Map(bottom=56007.0, center=[26.048206464191377, 90.60879118863485], controls=(WidgetControl(options=['position…

In [ ]:
import ee
import geemap

# Initialize Earth Engine
ee.Initialize()

# Define Goalpara District boundary
goalpara = ee.FeatureCollection('FAO/GAUL/2015/level2') \
    .filter(ee.Filter.eq('ADM2_NAME', 'Goalpara'))

# Function to generate and return the flood map for a given year
def get_flood_map(year):
    # Pre-Flood period (before monsoon)
    pre_flood = ee.ImageCollection('COPERNICUS/S1_GRD') \
        .filterDate(f'{year}-06-01', f'{year}-06-30') \
        .filterBounds(goalpara) \
        .filter(ee.Filter.eq('instrumentMode', 'IW')) \
        .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \
        .select('VV') \
        .median()

    # Post-Flood period (during or after monsoon)
    post_flood = ee.ImageCollection('COPERNICUS/S1_GRD') \
        .filterDate(f'{year}-07-01', f'{year}-09-30') \
        .filterBounds(goalpara) \
        .filter(ee.Filter.eq('instrumentMode', 'IW')) \
        .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \
        .select('VV') \
        .median()

    # Calculate flood extent (difference between pre and post flood)
    flood_map = pre_flood.subtract(post_flood).rename('flood_extent')

    # Apply threshold for flooded areas (fine-tune if necessary)
    flooded_areas = flood_map.gt(-2).updateMask(flood_map.gt(-2))

    # Mask everything outside the Goalpara district boundary
    flooded_areas = flooded_areas.clip(goalpara)

    return flooded_areas

# Loop through each year and create individual maps
for year in range(2015, 2024):
    flood_map = get_flood_map(year)

    # Create a new Map object for each year
    Map = geemap.Map(center=[26.17, 90.63], zoom=9)
    Map.addLayer(flood_map, {'palette': ['blue']}, f'Flooded Areas {year}')
    Map.addLayer(goalpara, {}, 'Goalpara District')
    Map.addLayerControl()  # Add layer control for interactivity

    # Display the map for the current year
    print(f"Flood Map for Year {year}")
    display(Map)


Flood Map for Year 2015


Map(center=[26.17, 90.63], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGU…

Flood Map for Year 2016


Map(center=[26.17, 90.63], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGU…

Flood Map for Year 2017


...

Map(center=[26.17, 90.63], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGU…

Flood Map for Year 2022


Map(center=[26.17, 90.63], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGU…

Flood Map for Year 2023


Map(center=[26.17, 90.63], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGU…

In [ ]:
# Import necessary modules
import ee
import geemap

# Authenticate and initialize Earth Engine
ee.Authenticate()  # Skip if already authenticated
ee.Initialize()

# Define the region of interest: Goalpara District
goalpara = ee.FeatureCollection("FAO/GAUL/2015/level2") \
    .filter(ee.Filter.eq('ADM2_NAME', 'Goalpara'))

# Function to mask non-flooded areas based on VV band threshold
def mask_water(image):
    vv = image.select('VV')
    flood = vv.lt(-18)  # Adjust threshold if necessary
    return image.updateMask(flood).clip(goalpara)

# Function to retrieve flood data for a specific year and month
def get_flood_image(year, month):
    start = ee.Date.fromYMD(year, month, 1)
    end = start.advance(1, 'month')

    collection = ee.ImageCollection('COPERNICUS/S1_GRD') \
        .filter(ee.Filter.date(start, end)) \
        .filter(ee.Filter.eq('instrumentMode', 'IW')) \
        .filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING')) \
        .filterBounds(goalpara) \
        .select('VV')

    flood_image = collection.median()  # Use median composite to reduce noise
    return mask_water(flood_image)

# Retrieve flood data for all years (August)
years = list(range(2015, 2024))  # From 2015 to 2023
flood_images = {year: get_flood_image(year, 8) for year in years}

# Create an interactive map
Map = geemap.Map()
Map.centerObject(goalpara, 10)

# Add Goalpara district boundary to the map
Map.addLayer(goalpara, {}, "Goalpara District")

# Define visualization parameters for the flood layers
flood_vis = {
    'min': -25,
    'max': 0,
    'palette': ['#0000FF', '#0080FF', '#00FFFF', '#00FF80', '#00FF00']  # Various shades of blue/green for flood extent
}

# Loop through the years and add corresponding flood layers
for year in years:
    flood_image = flood_images[year]

    # Add flood extent layer for each year
    Map.addLayer(
        flood_image, flood_vis, f'Flood Extent: {year}'
    )

# Define a legend for the flood extent
legend_items = [
    ("High Flood", "#0000FF"),   # Dark blue
    ("Medium Flood", "#0080FF"),  # Light blue
    ("Low Flood", "#00FFFF"),    # Cyan
    ("Very Low Flood", "#00FF80"), # Greenish cyan
    ("No Flood", "#00FF00")      # Green
]

# Add the legend to the map
Map.add_legend(
    title="Flood Extent Legend",
    legend_dict=dict(legend_items),  # Convert the list of tuples to a dictionary
    position="bottomright"
)

# Display the map
Map


Map(center=[26.048206464191377, 90.60879118863485], controls=(WidgetControl(options=['position', 'transparent_…

In [ ]:
# Import necessary modules
import ee
import geemap

# Authenticate and initialize Earth Engine
ee.Authenticate()  # Skip if already authenticated
ee.Initialize()

# Define the region of interest: Goalpara District
goalpara = ee.FeatureCollection("FAO/GAUL/2015/level2") \
    .filter(ee.Filter.eq('ADM2_NAME', 'Goalpara'))

# Function to mask non-flooded areas based on VV band threshold
def mask_water(image):
    vv = image.select('VV')
    flood = vv.lt(-18)  # Adjust threshold if necessary
    return image.updateMask(flood).clip(goalpara)

# Function to retrieve flood data for a specific year and month
def get_flood_image(year, month):
    start = ee.Date.fromYMD(year, month, 1)
    end = start.advance(1, 'month')

    collection = ee.ImageCollection('COPERNICUS/S1_GRD') \
        .filter(ee.Filter.date(start, end)) \
        .filter(ee.Filter.eq('instrumentMode', 'IW')) \
        .filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING')) \
        .filterBounds(goalpara) \
        .select('VV')

    flood_image = collection.median()  # Use median composite to reduce noise
    return mask_water(flood_image)

# Retrieve flood data for all required years (August)
years = [2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]
flood_images = {year: get_flood_image(year, 8) for year in years}

# Create an interactive map
Map = geemap.Map()
Map.centerObject(goalpara, 10)

# Add Goalpara district boundary to the map
Map.addLayer(goalpara, {}, "Goalpara District")

# Define visualization parameters for flood differences
diff_vis = {
    'min': -5,
    'max': 5,
    'palette': ['#00FF00', '#808080', '#FF0000']  # Green, Gray, Red
}

# Loop through consecutive years to add flood difference layers
for i in range(len(years) - 1):
    year1 = years[i]
    year2 = years[i + 1]

    # Get flood images for consecutive years
    flood_1 = flood_images[year1]
    flood_2 = flood_images[year2]

    # Calculate the difference between consecutive years
    flood_difference = flood_2.subtract(flood_1)

    # Add flood difference layer to the map
    Map.addLayer(
        flood_difference, diff_vis, f'Flood Difference: {year2} vs {year1}'
    )

# Define the legend items with hex color codes
legend_items = [
    ("Increased Flooding", "#FF0000"),  # Red
    ("No Change", "#808080"),           # Gray
    ("Decreased Flooding", "#00FF00")   # Green
]

# Add the legend to the map
Map.add_legend(
    title="Flood Difference Legend",
    legend_dict=dict(legend_items),  # Convert the list of tuples to a dictionary
    position="bottomright"
)

# Display the map
Map


Map(center=[26.048206464191377, 90.60879118863485], controls=(WidgetControl(options=['position', 'transparent_…

In [ ]:
# Import necessary modules
import ee
import geemap

# Authenticate and initialize Earth Engine
ee.Authenticate()  # Skip if already authenticated
ee.Initialize()

# Define the region of interest: Goalpara District
goalpara = ee.FeatureCollection("FAO/GAUL/2015/level2") \
    .filter(ee.Filter.eq('ADM2_NAME', 'Goalpara'))

# Function to mask non-flooded areas based on VV band threshold
def mask_water(image):
    vv = image.select('VV')
    flood = vv.lt(-18)  # Adjust threshold if necessary
    return image.updateMask(flood).clip(goalpara)

# Function to retrieve flood data for a specific year and month
def get_flood_image(year, month):
    start = ee.Date.fromYMD(year, month, 1)
    end = start.advance(1, 'month')

    collection = ee.ImageCollection('COPERNICUS/S1_GRD') \
        .filter(ee.Filter.date(start, end)) \
        .filter(ee.Filter.eq('instrumentMode', 'IW')) \
        .filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING')) \
        .filterBounds(goalpara) \
        .select('VV')

    flood_image = collection.median()  # Use median composite to reduce noise
    return mask_water(flood_image)

# Retrieve pre-flood (June) and post-flood (August) data for all years
years = list(range(2015, 2024))  # From 2015 to 2023
pre_flood_images = {year: get_flood_image(year, 6) for year in years}  # June as pre-flood
post_flood_images = {year: get_flood_image(year, 8) for year in years}  # August as post-flood

# Create an interactive map
Map = geemap.Map()
Map.centerObject(goalpara, 10)

# Add Goalpara district boundary to the map
Map.addLayer(goalpara, {}, "Goalpara District")

# Define distinct visualization parameters for pre-flood and post-flood
pre_flood_vis = {
    'min': -25,
    'max': 0,
    'palette': ['#00FF00', '#006400']  # Green shades for pre-flood (dry condition)
}

post_flood_vis = {
    'min': -25,
    'max': 0,
    'palette': ['#0000FF', '#00008B']  # Blue shades for post-flood (wet condition)
}

# Loop through years and add pre-flood (June) and post-flood (August) layers for each year
for year in years:
    pre_flood = pre_flood_images[year]
    post_flood = post_flood_images[year]

    # Add pre-flood (June) layers with green color palette
    Map.addLayer(pre_flood, pre_flood_vis, f'Pre-flood (June): {year}')

    # Add post-flood (August) layers with blue color palette
    Map.addLayer(post_flood, post_flood_vis, f'Post-flood (August): {year}')

# Define the legend for flood visualization
legend_items = [
    ("No Water (Pre-flood)", "#00FF00"),  # Light green for pre-flood (no water)
    ("Water/Flooded (Pre-flood)", "#006400"),  # Dark green for pre-flood (water presence)
    ("No Water (Post-flood)", "#0000FF"),  # Light blue for post-flood (no water)
    ("Water/Flooded (Post-flood)", "#00008B")  # Dark blue for post-flood (water presence)
]

# Add the legend to the map
Map.add_legend(
    title="Pre-flood and Post-flood Conditions",
    legend_dict=dict(legend_items),  # Convert the list of tuples to a dictionary
    position="bottomright"
)

# Display the map
Map


Map(center=[26.048206464191377, 90.60879118863485], controls=(WidgetControl(options=['position', 'transparent_…

In [ ]:
# Define Goalpara boundary using FAO GAUL dataset
goalpara = ee.FeatureCollection('FAO/GAUL/2015/level2') \
              .filter(ee.Filter.eq('ADM2_NAME', 'Goalpara')) \
              .filter(ee.Filter.eq('ADM1_NAME', 'Assam'))

# Calculate total area of Goalpara district
goalpara_area = goalpara.geometry().area().divide(1e6)  # Area in square kilometers
print('Total Area of Goalpara District (sq km):', goalpara_area.getInfo())


Total Area of Goalpara District (sq km): 1904.605647162269


In [ ]:
# Function to calculate the area of water bodies for a specific year
def calculate_water_area(year):
    start_date = f'{year}-06-01'
    end_date = f'{year}-10-31'

    # Sentinel-1 Image Collection (SAR) for water detection
    sentinel1 = ee.ImageCollection('COPERNICUS/S1_GRD') \
                  .filterBounds(goalpara) \
                  .filterDate(start_date, end_date) \
                  .filter(ee.Filter.eq('instrumentMode', 'IW')) \
                  .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \
                  .select('VV') \
                  .mean()

    # Define a threshold for detecting water
    water_mask = sentinel1.lt(-15)  # Adjust threshold if necessary

    # Calculate water area within Goalpara district
    water_area = water_mask.multiply(ee.Image.pixelArea()).reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=goalpara.geometry(),
        scale=30,
        maxPixels=1e10
    ).get('VV')

    water_area = ee.Number(water_area).divide(1e6)  # Area in sq km
    return water_area

# List to store water area for each year
years = list(range(2015, 2023))
water_areas = [calculate_water_area(year).getInfo() for year in years]

# Display water areas for each year
for year, water_area in zip(years, water_areas):
    print(f'Water Area in {year} (sq km): {water_area}')


Water Area in 2015 (sq km): 256.8716641591035
Water Area in 2016 (sq km): 245.03897278570017
Water Area in 2017 (sq km): 258.9282522188722
Water Area in 2018 (sq km): 229.99062939392329
Water Area in 2019 (sq km): 231.09460199587775
Water Area in 2020 (sq km): 342.19891856688463
Water Area in 2021 (sq km): 236.3666940420102
Water Area in 2022 (sq km): 232.26326382080168


In [ ]:
# Function to calculate flooded area based on pre- and post-flood season comparison
def calculate_flooded_area(year):
    # Define pre-flood and post-flood periods
    pre_flood = ee.ImageCollection('COPERNICUS/S1_GRD') \
                  .filterBounds(goalpara) \
                  .filterDate(f'{year}-01-01', f'{year}-05-31') \
                  .filter(ee.Filter.eq('instrumentMode', 'IW')) \
                  .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \
                  .select('VV') \
                  .mean()

    post_flood = ee.ImageCollection('COPERNICUS/S1_GRD') \
                   .filterBounds(goalpara) \
                   .filterDate(f'{year}-06-01', f'{year}-10-31') \
                   .filter(ee.Filter.eq('instrumentMode', 'IW')) \
                   .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \
                   .select('VV') \
                   .mean()

    # Apply water detection thresholds
    pre_flood_water = pre_flood.lt(-15)
    post_flood_water = post_flood.lt(-15)

    # Identify flooded areas as areas that are water post-flood but not pre-flood
    flooded_area_mask = post_flood_water.And(post_flood_water.neq(pre_flood_water))

    # Calculate flooded area
    flooded_area = flooded_area_mask.multiply(ee.Image.pixelArea()).reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=goalpara.geometry(),
        scale=30,
        maxPixels=1e10
    ).get('VV')

    flooded_area = ee.Number(flooded_area).divide(1e6)  # Area in sq km
    return flooded_area

# List to store flooded area for each year
flooded_areas = [calculate_flooded_area(year).getInfo() for year in years]

# Display flooded areas for each year
for year, flooded_area in zip(years, flooded_areas):
    print(f'Flooded Area in {year} (sq km): {flooded_area}')


Flooded Area in 2015 (sq km): 85.33665757677338
Flooded Area in 2016 (sq km): 47.75934485817129
Flooded Area in 2017 (sq km): 64.91462517014037
Flooded Area in 2018 (sq km): 42.66528270702215
Flooded Area in 2019 (sq km): 56.61636854936162
Flooded Area in 2020 (sq km): 149.89436958681256
Flooded Area in 2021 (sq km): 43.7873447211591
Flooded Area in 2022 (sq km): 51.68734703779414
